# E-Mail classification NLP (step 3)

## Text classification data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#from nltk.stem import WordNetLemmatizer

from collections import Counter
import re
import string


#url_testDATA='https://drive.google.com/file/d/1q8_G19zYuD4C2l8u4CrGe_yiitjI1h2I/view?usp=sharing'
url_trainDATA='https://drive.google.com/file/d/1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW/view?usp=sharing'

#https://www.kaggle.com/datatattle/email-classification-nlp?select=SMS_train.csv
url=url_trainDATA
file_id=url.split('/')[-2]
print('LOG file id = ',file_id)
dwn_url='https://drive.google.com/uc?id=' + file_id
print('LOG download url = ', dwn_url)

df = pd.read_csv(dwn_url, encoding = "ISO-8859-1")

LOG file id =  1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW
LOG download url =  https://drive.google.com/uc?id=1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW


In [2]:
df.head()

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam


In [3]:
df.drop(['S. No.'], axis = 1, inplace = True)

In [4]:
X_train = df.Message_body
Y_train = df.Label

print("\nTEXT OF THE MESSAGE:\n",  X_train)
print("\nCLASSIFICATION:\n", Y_train)


TEXT OF THE MESSAGE:
 0                             Rofl. Its true to its name
1      The guy did some bitching but I acted like i'd...
2      Pity, * was in mood for that. So...any other s...
3                   Will ü b going to esplanade fr home?
4      This is the 2nd time we have tried 2 contact u...
                             ...                        
952    hows my favourite person today? r u workin har...
953                        How much you got for cleaning
954    Sorry da. I gone mad so many pending works wha...
955                                   Wat time ü finish?
956                      Just glad to be talking to you.
Name: Message_body, Length: 957, dtype: object

CLASSIFICATION:
 0      Non-Spam
1      Non-Spam
2      Non-Spam
3      Non-Spam
4          Spam
         ...   
952    Non-Spam
953    Non-Spam
954    Non-Spam
955    Non-Spam
956    Non-Spam
Name: Label, Length: 957, dtype: object


In [5]:
df = df[(df.Label == 'Non-Spam') | (df.Label == 'Spam')]
print(df)

                                          Message_body     Label
0                           Rofl. Its true to its name  Non-Spam
1    The guy did some bitching but I acted like i'd...  Non-Spam
2    Pity, * was in mood for that. So...any other s...  Non-Spam
3                 Will ü b going to esplanade fr home?  Non-Spam
4    This is the 2nd time we have tried 2 contact u...      Spam
..                                                 ...       ...
952  hows my favourite person today? r u workin har...  Non-Spam
953                      How much you got for cleaning  Non-Spam
954  Sorry da. I gone mad so many pending works wha...  Non-Spam
955                                 Wat time ü finish?  Non-Spam
956                    Just glad to be talking to you.  Non-Spam

[957 rows x 2 columns]


In [6]:
def Sort_Sp_NSp(text):
    if text in ['Non-Spam']:
        return 'important_message'
    elif text in ['Spam']:
        return 'spammmm'
    else:
        return text

In [7]:
df.Label = df.Label.apply(Sort_Sp_NSp)
print(df.Label)

0      important_message
1      important_message
2      important_message
3      important_message
4                spammmm
             ...        
952    important_message
953    important_message
954    important_message
955    important_message
956    important_message
Name: Label, Length: 957, dtype: object


In [8]:
X_train = df.Message_body
Y_train = df.Label

In [9]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words("english")
def clean_text(text):
    text = text.lower()           # converting to lowercase
    text = re.sub("@\S+", " ", text)  
    text = re.sub("https*\S+", " ", text)      #\S - не пробел 
    text = re.sub("www\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\d", " ", text)        # remove all numbers
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)     # remove punctuations
    text = re.sub('\n', ' ', text)        
    text = re.sub('\s{2,}',' ', text)        # remove extra spaces
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

In [10]:
def digital_label(text):
    if text == 'important_message':
        return 1.0
    elif text == 'spammmm':
        return 0.0

In [11]:
X_train = X_train.apply(clean_text)
Y_train = Y_train.apply(digital_label)
print(X_train)
print(Y_train)

0                                         rofl true name
1      guy bitching acted like interested buying some...
2                                 pity mood suggestions 
3                           ü b going esplanade fr home 
4      nd time tried contact u u £ pound prize claim ...
                             ...                        
952    hows favourite person today r u workin hard sl...
953                                    much got cleaning
954                sorry da gone mad many pending works 
955                                   wat time ü finish 
956                                        glad talking 
Name: Message_body, Length: 957, dtype: object
0      1.0
1      1.0
2      1.0
3      1.0
4      0.0
      ... 
952    1.0
953    1.0
954    1.0
955    1.0
956    1.0
Name: Label, Length: 957, dtype: float64


In [12]:
whole_text = ""
for i in X_train:
    whole_text += i + ' '

In [13]:
tokens = word_tokenize(whole_text)
lemmatizer = WordNetLemmatizer()
lemtok = [lemmatizer.lemmatize(t) for t in tokens]
dictionary = list(set(lemtok))

In [14]:
len(dictionary)

2638

In [15]:
def convert(text):
    tok = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(t) for t in tok]
    res = [dictionary.index(i) for i in lem]
    return res

In [16]:
X_train = X_train.apply(convert)
print(X_train)

0                                      [1113, 408, 2105]
1      [268, 883, 368, 2334, 2019, 1643, 1738, 2463, ...
2                                      [1279, 1285, 356]
3                    [2318, 2284, 406, 2150, 2035, 2182]
4      [1026, 1499, 1067, 1991, 1684, 1684, 1610, 160...
                             ...                        
952    [2520, 2445, 1031, 2410, 732, 1684, 1530, 1933...
953                                    [132, 1158, 1681]
954               [606, 103, 547, 2027, 2271, 1677, 753]
955                             [1339, 1499, 2318, 1481]
956                                           [1718, 36]
Name: Message_body, Length: 957, dtype: object


In [23]:
def vectorize_sequences(sequences, dimension = 30000):#dims
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers
#import matplotlib.pyplot as plt

x_train = vectorize_sequences(X_train)
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [25]:
y_train = np.asarray(Y_train).astype("float32")

#print(y_train)

In [27]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers



max_seq_len = 80
max_features = 30000
x_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = max_seq_len)
emb_dim = 128

model = keras.Sequential()
model.add(layers.Embedding(max_features, emb_dim))
model.add(layers.LSTM(128))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [28]:
model.compile(optimizer = "adam",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

history = model.fit(x_train,
                    y_train,
                    epochs = 10,
                    batch_size = 256,
                    validation_split = 0.2, 
                    shuffle = True, callbacks = my_callbacks, 
                    verbose = 1)

Epoch 1/10
3/3 [==============================] - 7s 1s/step - loss: 0.6683 - accuracy: 0.8183 - val_loss: 0.6105 - val_accuracy: 0.8802
Epoch 2/10
3/3 [==============================] - 2s 773ms/step - loss: 0.5575 - accuracy: 0.8706 - val_loss: 0.3978 - val_accuracy: 0.8802
Epoch 3/10
3/3 [==============================] - 2s 800ms/step - loss: 0.3657 - accuracy: 0.8706 - val_loss: 0.3636 - val_accuracy: 0.8802
Epoch 4/10
3/3 [==============================] - 2s 854ms/step - loss: 0.3602 - accuracy: 0.8706 - val_loss: 0.2920 - val_accuracy: 0.8802
Epoch 5/10
3/3 [==============================] - 2s 785ms/step - loss: 0.2931 - accuracy: 0.8706 - val_loss: 0.2946 - val_accuracy: 0.8802
Epoch 6/10
3/3 [==============================] - 2s 835ms/step - loss: 0.2854 - accuracy: 0.8706 - val_loss: 0.2662 - val_accuracy: 0.8802
Epoch 7/10
3/3 [==============================] - 2s 817ms/step - loss: 0.2378 - accuracy: 0.8758 - val_loss: 0.2186 - val_accuracy: 0.8802
Epoch 8/10
3/3 [=======

In [29]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         3840000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 3,971,713
Trainable params: 3,971,713
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 0.09465945512056351
Test accuracy: 0.9843260049819946
